In [ ]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง library ที่จำเป็น
!pip install -q transformers torch datasets accelerate

import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm.auto import tqdm

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


In [ ]:
# %% 2. การเตรียมข้อมูล

print("กำลังโหลด Wongnai Reviews Dataset...")
try:
    dataset = load_dataset("wongnai/wongnai_reviews", split='train')
    df = dataset.to_pandas()
    print(f"โหลดข้อมูลดิบสำเร็จ จำนวน {len(df)} รีวิว")

    # สร้างข้อมูล test โดยสุ่มมา 500 ตัวอย่าง
    test_df = df.sample(n=500, random_state=42).reset_index(drop=True)
    test_df = test_df[['review_body']].rename(columns={'review_body': 'review_text'})
    test_df['review_id'] = test_df.index

    print("\nสร้าง Test Set สำหรับทดลอง Pipeline สำเร็จ")
    print("ตัวอย่างข้อมูล:")
    print(test_df.head())

except Exception as e:
    print(f"ไม่สามารถโหลด Dataset ได้: {e}")
    # สร้างข้อมูลสำรองหากโหลดไม่ได้
    dummy_data = {
        'review_id': [1, 2, 3],
        'review_text': [
            "อาหารอร่อยมากครับ โดยเฉพาะกุ้งเผา แต่รอนานไปหน่อย ราคาไม่แพง",
            "รสชาติเฉยๆนะ แต่ร้านสวยมาก บรรยากาศดี",
            "ร้านหาง่าย เดินทางสะดวก"
        ]
    }
    test_df = pd.DataFrame(dummy_data)
    print("\nสร้าง Test Set ตัวอย่างขึ้นมาแทน:")
    print(test_df)


In [ ]:
# %% 3. การโหลดโมเดล Qwen2 และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"

print(f"กำลังโหลด Tokenizer จาก '{model_id}'...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f"กำลังโหลดโมเดล '{model_id}'...")
print("หมายเหตุ: ขั้นตอนนี้ต้องการ VRAM ประมาณ 14-16 GB")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

print("\nโหลดโมเดลและ Tokenizer สำเร็จ!")


In [ ]:
# %% 4. การออกแบบ Prompt สำหรับ Qwen2

# สร้างโครงสร้าง Prompt แบบ Chat ที่มีประสิทธิภาพสำหรับ Qwen2
# 1. System Prompt: สั่งบทบาทและรูปแบบผลลัพธ์ที่ชัดเจนที่สุด
# 2. Few-Shot Examples: ใส่ตัวอย่างที่ดี 3 กรณี (ผสม, เฉพาะทาง, และไม่พบ)
# 3. Final User Prompt: คำถามสุดท้ายที่เราจะใส่รีวิวเข้าไป

chat_template = [
    {
        "role": "system",
        "content": "คุณคือผู้เชี่ยวชาญในการวิเคราะห์รีวิว หน้าที่ของคุณคือสกัด 'aspect' และ 'sentiment' จากรีวิวที่กำหนด และต้องตอบกลับเป็น JSON Array เท่านั้น ห้ามมีข้อความอธิบายอื่นใดๆ ถ้าไม่พบ aspect ที่เกี่ยวข้อง ให้ตอบกลับเป็น Array ว่าง `[]`"
    },
    {
        "role": "user",
        "content": """รีวิว: "อาหารอร่อยมาก แต่ราคาแอบแรงไปนิดนึง บรรยากาศร้านดีครับ" จงสกัดแง่มุมและความรู้สึก"""
    },
    {
        "role": "assistant",
        "content": """[{"aspect": "รสชาติ", "sentiment": "positive"}, {"aspect": "ราคา", "sentiment": "negative"}, {"aspect": "บรรยากาศ", "sentiment": "positive"}]"""
    },
    {
        "role": "user",
        "content": """รีวิว: "รสชาติโอเค พอใช้ได้ แต่พนักงานบริการช้ามาก" จงสกัดแง่มุมและความรู้สึก"""
    },
    {
        "role": "assistant",
        "content": """[{"aspect": "รสชาติ", "sentiment": "neutral"}, {"aspect": "บริการ", "sentiment": "negative"}]"""
    },
    {
        "role": "user",
        "content": """รีวิว: "ร้านหาง่ายดี เดินทางสะดวก" จงสกัดแง่มุมและความรู้สึก"""
    },
    {
        "role": "assistant",
        "content": """[]"""
    },
    {
        "role": "user",
        "content": """รีวิว: "{review_text}" จงสกัดแง่มุมและความรู้สึก"""
    }
]

# ใช้ .apply_chat_template เพื่อสร้าง prompt ที่สมบูรณ์ตามรูปแบบของ Qwen2
# นี่คือวิธีที่ถูกต้องและให้ผลดีที่สุด
prompt_example = tokenizer.apply_chat_template(
    chat_template[:-1], # แสดงตัวอย่าง prompt ที่ไม่มี review จริง
    tokenize=False,
    add_generation_prompt=True
)

print("--- ตัวอย่าง Prompt ที่จะส่งให้โมเดล (ก่อนเติมรีวิวจริง) ---")
print(prompt_example)


In [ ]:
# %% 5. การเริ่ม Pipeline การทำนายผล

results = []
# ในการทดลอง สามารถลดจำนวน test_subset ลงได้
test_subset = test_df

print(f"กำลังเริ่มทำนายผลสำหรับรีวิว {len(test_subset)} ตัวอย่าง...")

for index, row in tqdm(test_subset.iterrows(), total=len(test_subset)):
    review_id = row['review_id']
    review_text = row['review_text']

    # สร้าง Prompt สำหรับรีวิวปัจจุบัน
    current_chat = [
        *chat_template[:-1], # นำ template ทั้งหมดมาใช้
        {"role": "user", "content": chat_template[-1]["content"].format(review_text=review_text)}
    ]

    # แปลง prompt เป็น token id ตามรูปแบบของโมเดล
    inputs = tokenizer.apply_chat_template(
        current_chat,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(DEVICE)

    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=256,
            do_sample=False, # ไม่ต้องสุ่มคำตอบเพื่อให้ได้ผลลัพธ์ที่แน่นอน
            pad_token_id=tokenizer.eos_token_id
        )

    # ถอดรหัสเฉพาะส่วนที่โมเดลสร้างขึ้นใหม่
    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # --- ส่วนของการ Parse ผลลัพธ์ที่รัดกุม ---
    try:
        # ค้นหาตำแหน่งเริ่มต้น `[` และตำแหน่งสุดท้าย `]` ของ JSON
        start_index = response_text.find('[')
        end_index = response_text.rfind(']')
        
        # ถ้าหาเจอทั้งคู่ ให้ตัดมาเฉพาะส่วนของ JSON
        if start_index != -1 and end_index != -1:
            json_part = response_text[start_index : end_index + 1]
            parsed_output = json.loads(json_part)

            if isinstance(parsed_output, list):
                for item in parsed_output:
                    if isinstance(item, dict) and 'aspect' in item and 'sentiment' in item:
                        results.append({
                            'id': review_id,
                            'aspect': item['aspect'],
                            'sentiment': item['sentiment']
                        })
        # ถ้าหา JSON ไม่เจอ ก็จะข้ามไป (ซึ่งไม่ควรเกิดจาก prompt ที่ดี)

    except (json.JSONDecodeError, IndexError) as e:
        print(f"\n[Error] ไม่สามารถ Parse JSON สำหรับ review_id {review_id}.")
        print(f"  -> ผลลัพธ์จากโมเดล: '{response_text}'")

print("\nการทำนายผลเสร็จสิ้น")


In [ ]:
# %% 6. การสร้างไฟล์ Submission

print("กำลังสร้างไฟล์ submission.csv...")

if not results:
    print("ไม่พบผลลัพธ์ที่ถูกต้อง! ไม่สามารถสร้างไฟล์ submission ได้")
else:
    submission_df = pd.DataFrame(results)
    submission_df = submission_df[['id', 'aspect', 'sentiment']]

    print("\nตัวอย่างผลลัพธ์ 10 แถวแรก:")
    print(submission_df.head(10))

    submission_df.to_csv("submission.csv", index=False)
    print("\nสร้างไฟล์ submission.csv สำเร็จ!")
